In [ ]:
"""
Link to kaggle notebook:
https://www.kaggle.com/parthvora/3d-cnn-predicting-mask-to-genomic-features/

"""

In [ ]:
import os
from glob import glob
import random

from collections import OrderedDict
import numpy as np
import pandas as pd
import cv2 as cv
from PIL import Image
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF

In [ ]:
df = pd.read_csv("/kaggle/input/lgg-mri-segmentation/lgg-mri-segmentation/kaggle_3m/data.csv")
labels = df.COCCluster.to_numpy()
labels -= 1
print(np.unique(labels))

patients = []
data_dir = '/kaggle/input/lgg-mri-segmentation/kaggle_3m/'
for dirname, _, filenames in os.walk(data_dir):
    if dirname == data_dir:
        continue
        
    patients.append(dirname)

#sort(patients)

coc = []
coc.append(len(np.where(labels == 0)[0]))
coc.append(len(np.where(labels == 1)[0]))
coc.append(len(np.where(labels == 2)[0]))

plt.bar([1, 2, 3], coc)
plt.xlabel("Type")
plt.xticks([1, 2, 3])
plt.ylabel("Num Patients")
plt.title("Histogram of patients for COC subtype")

In [ ]:
class MRIDataset3D(Dataset):
    
    # Takes in a list of images
    def __init__(self, patients, labels):
        self.patients = patients
        self.labels = labels
    
    def __len__(self):
        return(len(self.patients))
    
    def __getitem__(self, i):

        patient = self.patients[i]
        label = self.labels[i]
        
        files = os.listdir(patient)
        
        # 1 channel, max of 88 frames
        masks = np.zeros((1, 88, 256, 256))
        
        # Stack masks together to make a 3D box
        for i in files:
            if "mask" in i:
                fn = patient + "/" + i
                m = cv.imread(fn, cv.IMREAD_GRAYSCALE)               
                index = int(i.split("_")[-2]) - 1
                masks[0, index, :, :] = m

        return masks, label

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(patients, labels, test_size=0.2, shuffle=True, stratify=labels) 

print(len(y_test))

train_dataset = MRIDataset3D(X_train, y_train)
test_dataset = MRIDataset3D(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

for (i, (x, y)) in enumerate(train_loader):
    print(x.dtype)
    break


In [ ]:
class CNN3D(nn.Module):
    def __init__(self):
        super(CNN3D, self).__init__()
        
        self.layer1 = nn.Sequential(
                        nn.Conv3d(in_channels=1, out_channels=2, kernel_size=7, stride=2), 
                        nn.ReLU(),
                        nn.MaxPool3d(kernel_size=3, stride=2)
                      )
        
        self.layer2 = nn.Sequential(
                          nn.Conv3d(in_channels=2, out_channels=4, kernel_size=5, stride=1), 
                          nn.ReLU(),
                          nn.MaxPool3d(kernel_size=3, stride=2)
                      )
        
        self.layer3 = nn.Sequential(
                          nn.Conv3d(in_channels=4, out_channels=8, kernel_size=3, stride=1), 
                          nn.ReLU(),
                          nn.MaxPool3d(kernel_size=3, stride=2)
                      )
        
        self.linear1 = nn.Sequential(
                           nn.Linear(8 * 2 * 12 * 12, 100),
                           nn.ReLU()
                       )
        
        #self.linear2 = nn.Sequential(
        #                   nn.Linear(1000, 100),
        #                   nn.ReLU()
        #               )
        
        self.linear3 = nn.Sequential(
                           nn.Linear(100, 3),
                       )
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)

        out = self.linear1(out)
        nn.Dropout(0.5)
        #out = self.linear2(out)
        #nn.Dropout(0.5)
        out = self.linear3(out)

        return out

In [ ]:
def train(model, train_loader, test_loader, epochs, lr):
    
    # Can try different optimizers
    optimizer = torch.optim.Adam(model.parameters(), lr)
    compute_loss = nn.CrossEntropyLoss()
    
    train_loss, test_loss = [], []
    
    if torch.cuda.is_available():
        model = model.cuda()
    else:
        print("ERROR: CUDA NOT AVAILABLE")
        return [], []
    
    for epoch in range(epochs):
        
        # Train
        for step, (X, y) in enumerate(train_loader):
            
            optimizer.zero_grad()
            
            X = X.float()           
            X = X.cuda()
                
            y_hat = model(X)
            y_hat = y_hat.cpu()
            
            loss = compute_loss(y_hat, y)
            
            train_loss.append(loss.item())
            
            loss.backward()
            optimizer.step()

            del X, y, y_hat
            
        # Test
        t = []
        for step, (X, y) in enumerate(test_loader):
            
            X = X.float()
            X = X.cuda()
                
            y_hat = model(X)
            y_hat = y_hat.cpu()
            
            loss = compute_loss(y_hat, y)
            t.append(loss.item())
            
            del X, y, y_hat
        test_loss.append(np.mean(t))
        
        print("Epoch: {0}".format(epoch + 1))
        print("Train Loss: {0}".format(train_loss[-1]))
        print("Validation Loss: {0}".format(test_loss[-1]))
     
    return train_loss, test_loss

In [ ]:
model = CNN3D()

epochs = 10
lr = 0.00005
train_loss, test_loss = train(model, train_loader, test_loader, epochs, lr)

In [ ]:
plt.plot(range(len(train_loss)), train_loss)
plt.plot([0, 10, 20, 30, 40, 50, 60, 70, 80, 90], test_loss)
plt.xticks([0, 10, 20, 30, 40, 50, 60, 70, 80, 90], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
plt.xlabel("Epochs")
plt.ylabel("Cross Entropy Loss")
plt.title("Training and Validation Loss for 3D CNN")
plt.legend(["Training", "Validation"])

In [ ]:
def test(model, test_loader):
    
    acc = []
    preds = []
    true_labels = []
    for step, (X, y) in enumerate(test_loader):
        
        X = X.float()
        X = X.cuda()
        
        y_hat = model(X)
        y_hat = y_hat.cpu()
        
        y_hat = y_hat.detach().numpy().reshape(3)
        pred = np.argmax(y_hat)
        y = y.numpy()

        preds.append(pred)
        true_labels.append(y)
        acc.append(pred == y)
        
    return acc, true_labels, preds
    
    

In [ ]:
acc, true_labels, preds = test(model, test_loader)
print(np.mean(acc))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
cm = confusion_matrix(true_labels, preds)
sn.heatmap(cm, annot=True, xticklabels=[1, 2, 3], yticklabels=[1, 2, 3])
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion matrix for 3D CNN (20% Test Set)")
